## **City Bike Initial Data Analysis (IDA)**

In this step we are going to dive into the dataset of Citibike riderships in February 2016 and do an initial data analysis in which we will:

* Explore the available variables we have for each observation
* Format the data columns to correct types, as well as check the data sanity of the date columns
* Explore the presence of missing data: how much, in which column and whether there is some pattern in it
* Add new helper columns and drop erroneous rows.

Finally, we are going to store our processed bike and weather datasets into new csv files for further processing and analysis.

In [1]:
# loading used packages
import pandas as pd
from utilities.date_utilities import is_bike_time_correct, format_bike_times
from utilities.preprocess import attach_additional_bike_columns, cast_bike_types
from os.path import abspath

# constants
RIDES_YEAR = 2016
RIDES_MONTH = 2
START_TIME_COL = 'starttime'
STOP_TIME_COL = 'stoptime'
TIME_COLUMNS = [START_TIME_COL, STOP_TIME_COL]

# constants
CITIBIKE_FILENAME = "../data/raw/201602-citibike-tripdata.csv"
WEATHER_FILENAME = "../data/raw/weather_data_nyc_centralpark_2016.csv"

# a filename of the dataset after preprocessing
WEATHERLY_BIKES_FILENAME = "../data/processed/weatherly_bikes_2016_02.csv"

### Loading and preparing the bike ridership data

In [2]:
# loading the bikes dataset
bikes_df = pd.read_csv(abspath(CITIBIKE_FILENAME))
bikes_df.head(2)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,461,2/1/2016 00:00:08,2/1/2016 00:07:49,480,W 53 St & 10 Ave,40.766697,-73.990617,524,W 43 St & 6 Ave,40.755273,-73.983169,23292,Subscriber,1966.0,1
1,297,2/1/2016 00:00:56,2/1/2016 00:05:53,463,9 Ave & W 16 St,40.742065,-74.004432,380,W 4 St & 7 Ave S,40.734011,-74.002939,15329,Subscriber,1977.0,1


####  Format columns, cast to correct types and add new helper ones. Dates sanity check

In [3]:
# checking the sanity of the dates of the bike riderships
starttime_correct = is_bike_time_correct(bikes_df['starttime']).mean() * 100
stoptime_correct = is_bike_time_correct(bikes_df['stoptime']).mean() * 100

print("The starttime column is correct in {p:.2f}% of the cases.".format(p=starttime_correct))
print("The stoptime column is correct in {p:.2f}% of the cases.".format(p=stoptime_correct))

The starttime column is correct in 100.00% of the cases.
The stoptime column is correct in 100.00% of the cases.


In [4]:
# cast start and stop time to correct formats
bikes_df[TIME_COLUMNS] = format_bike_times(bikes_df[TIME_COLUMNS])

# cast to correct types
bikes_df = cast_bike_types(bikes_df)

# add additional helper columns to help with the analysis
bikes_df = attach_additional_bike_columns(bikes_df)

# see the processed dataframe
bikes_df.head(3)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,gender,month,day,start_hour,start_minute,stop_hour,stop_minute,age,tripduration_minutes,tripduration_hours
0,461,2016-02-01 00:00:08,2016-02-01 00:07:49,480,W 53 St & 10 Ave,40.766697,-73.990617,524,W 43 St & 6 Ave,40.755273,...,male,2,1,0,0,0,7,50,7.683333,0.128056
1,297,2016-02-01 00:00:56,2016-02-01 00:05:53,463,9 Ave & W 16 St,40.742065,-74.004432,380,W 4 St & 7 Ave S,40.734011,...,male,2,1,0,0,0,5,39,4.950000,0.082500
2,280,2016-02-01 00:01:00,2016-02-01 00:05:40,3134,3 Ave & E 62 St,40.763126,-73.965269,3141,1 Ave & E 68 St,40.765005,...,male,2,1,0,1,0,5,29,4.666667,0.077778


#### Percentage of NAN values in each column

In [5]:
bikes_nan_props_ = bikes_df.isna().mean() * 100

# percentage of nan values in each column
bikes_nan_props_[bikes_nan_props_ > 0].map('{:,.2f}%'.format)

birth year    5.32%
age           5.32%
dtype: object

Only the column 'birth year' has missing values. That is of slightly above 5%. Let's see how missing age is distributed according to missing gender.

In [6]:
# cross tabulation with gender NA values looking for some pattern in missing data
year_missing_by_gender_df = pd.crosstab(bikes_df['gender'], bikes_df['birth year'].isna())
year_missing_by_gender_df.columns = ["Birth year known", "Birth year unknown"]

year_missing_by_gender_df

,Birth year known,Birth year unknown
gender,,
female,112587,0
male,417215,1
unknown,1246,29825


We can see that bikers with missing age, are having unknown gender as well. Maybe we can find some pattern with the different user types (Customer and Annual Subscriber)?

In [7]:
year_missing_by_usertype_df = pd.crosstab(bikes_df['usertype'], bikes_df['birth year'].isna())
year_missing_by_usertype_df.columns = ["Birth year known", "Birth year unknown"]

year_missing_by_usertype_df

,Birth year known,Birth year unknown
usertype,,
Customer,0,29825
Subscriber,531048,1


We have only one subscriber whose age is not available. The rest of the ones with unavailable age are all customers. One assumption for that might be that this information is not collected in any way for the customer users.

#### Exploring Birth year counts

In [8]:
pd.concat(
    [bikes_df['age'].value_counts(normalize=False),
     bikes_df['age'].value_counts(normalize=True)],
    axis=1,
    keys=('counts', 'percentage')
).sort_index(ascending=False)

,counts,percentage
131,94,0.000177
117,38,0.000072
116,101,0.00019
115,2,0.000004
109,4,0.000008
...,...,...
20,2971,0.005595
19,1229,0.002314
18,1363,0.002567
17,512,0.000964


We can see that there are 94 people of age 131 and others of age above 100 years. This is probably due to erroneous data.

##### Trip duration

In [9]:
print("Trips above 500 hours: ", (bikes_df['tripduration_hours'] > 500).sum())
print("Trips above 30 hours: ", (bikes_df['tripduration_hours'] > 30).sum())
print("----------------------------")
print("99th emperical quantile is {q:.2f} minutes".format(q=bikes_df['tripduration_minutes'].quantile(.99)))

Trips above 500 hours:  2
Trips above 30 hours:  202
----------------------------
99th emperical quantile is 51.79 minutes


Lets also do another cross tabulation to see whether longer rides are not correlated with the user types in some manner.

In [10]:
x = pd.crosstab(
    (bikes_df['tripduration_hours'] > 1).map({True: '> 1 hour', False: '< 1 hour'}),
    bikes_df['usertype'],
    margins=True
)
# (2509 / 29825) * 100 :- percentage of customers riding above an hour
# (2024 / 531049) * 100 :- percentage of subscribers riding above an hour


There are 202 trips lasting more than 30 hours, and even some lasting longer than 500 hours. However, 99% of <u>all</u> the trips' duration is below an hour.

Aproximately 8% of the <u>customer</u> type trips have lasted more than an hour, while the <u>subscribers'</u> trips lasting more than an hour are only 0.4% (less than percent). <br/>
One assumption for that might be that some riders, especially customers, are forgetting to dock their bikes after usage. We should keep in mind that fact, since those observations would greatly affect the mean of the distribution of the trip durations.

### Dropping erroneous data

In [11]:
# dropping with age above 90
MAX_AGE = 90

drop_mask = bikes_df['age'] > MAX_AGE
print("We are going to drop {p:.2f}% of our data".format(p=drop_mask.mean() * 100))

bikes_df = bikes_df[(~drop_mask | bikes_df['age'].isna())]

We are going to drop 0.05% of our data


### **Joining our data with the weather data**

In [12]:
weather_df = pd.read_csv(WEATHER_FILENAME)
weather_df.head(5)

,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,1-1-2016,42,34,38.0,0.00,0.0,0
1,2-1-2016,40,32,36.0,0.00,0.0,0
2,3-1-2016,45,35,40.0,0.00,0.0,0
3,4-1-2016,36,14,25.0,0.00,0.0,0
4,5-1-2016,29,11,20.0,0.00,0.0,0


Next, we extract the weather rows only for Febuary, do some date reformatting and add new columns to the weather dataset.

In [13]:
# format the weather date to a correct format and join with our dataset
weather_df['date'] = weather_df['date'].apply(pd.to_datetime, format="%d-%m-%Y")

# add additional columns
weather_df['month'] = weather_df['date'].dt.month
weather_df['year'] = weather_df['date'].dt.year
weather_df['day'] = weather_df['date'].dt.day

# drop weather data which is not in february
feb_mask = (weather_df['month'] == RIDES_MONTH) & (weather_df['year'] == RIDES_YEAR)
weather_df = weather_df[feb_mask]
weather_df.head()

,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,month,year,day
31,2016-02-01,59,44,51.5,0.01,0.0,2,2,2016,1
32,2016-02-02,50,38,44.0,0.00,0.0,T,2,2016,2
33,2016-02-03,59,42,50.5,0.73,0.0,0,2,2016,3
34,2016-02-04,59,44,51.5,T,0.0,0,2,2016,4
35,2016-02-05,44,31,37.5,0.53,2.5,1,2,2016,5


Finally, we store our processed datasets to new csv files, so that we use them in our furher analysis.

In [14]:
# writing our processed dataframes to csvs
bikes_df.to_csv(abspath("../data/processed/bikes_2016_02.csv"))
weather_df.to_csv(abspath("../data/processed/weather_2016_02.csv"))

# weatherly_bikes_df = pd.merge(left=bikes_df, right=weather_df, left_on='day', right_on='day')
# weatherly_bikes_df.to_csv("../data/processed/weatherly_bikes_2016_02.csv")
# weatherly_bikes_df['average temperature'].describe()